In [8]:
# ============================================================
# Imports
# ============================================================

# Core Mesa library for agent-based modeling
import mesa
from mesa.time import RandomActivation

# Numerical and data manipulation
import numpy as np
import pandas as pd

# Visualization (if you need later)
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

print("Mesa version:", mesa.__version__)


# ============================================================
# HouseAgent – each dwelling is an agent
# ============================================================

class HouseAgent(mesa.Agent):
    def __init__(self, model, house_id, energy_label, market_value):
        # IMPORTANT for your Mesa version: only pass `model`
        super().__init__(model)

        # Our own fixed ID for this dwelling
        self.house_id = house_id
        self.energy_label = energy_label
        self.market_value = market_value
        self.owner_id = None  # unique_id of HomeownerAgent or None

    def is_vacant(self):
        return self.owner_id is None

    def step(self):
        # Houses are passive for now
        pass


# ============================================================
# HomeownerAgent – decision-making agent
# ============================================================

class HomeownerAgent(mesa.Agent):
    def __init__(
        self,
        model,
        current_house_id,
        financial_capacity,
        pref_energy,
        pref_value,
        min_acceptable_label,
    ):
        # Only pass `model`
        super().__init__(model)

        # State variables
        self.current_house_id = current_house_id
        self.financial_capacity = financial_capacity
        self.pref_energy = pref_energy
        self.pref_value = pref_value
        self.min_acceptable_label = min_acceptable_label

        # Indicators
        self.has_moved = 0
        self.has_upgraded = 0

        # Decision in {"stay", "stay_upgrade", "move", "move_upgrade"}
        self.decision_type = None

        # Internal info for chosen options
        self._stay_upgrade_info = None      # (new_label, cost)
        self._move_info = None              # target_house_id
        self._move_upgrade_info = None      # (house_id, new_label, cost)

    # ---------------- Helper methods ----------------

    def current_house(self):
        return self.model.houses[self.current_house_id]

    def norm_energy(self, house):
        el_min = self.model.energy_label_min
        el_max = self.model.energy_label_max
        if el_max == el_min:
            return 0.0
        return (house.energy_label - el_min) / float(el_max - el_min)

    def norm_value(self, house):
        mv_min = self.model.market_value_min
        mv_max = self.model.market_value_max
        if mv_max == mv_min:
            return 0.0
        return (house.market_value - mv_min) / float(mv_max - mv_min)

    def satisfaction(self, house, override_label=None):
        """
        Satisfaction = PrefEnergy * NormEnergyLabel + PrefValue * NormMarketValue

        If override_label is given, temporarily pretend the house has that
        label (for evaluating upgrades).
        """
        original_label = house.energy_label
        if override_label is not None:
            house.energy_label = override_label

        s = (
            self.pref_energy * self.norm_energy(house)
            + self.pref_value * self.norm_value(house)
        )

        # restore original label
        house.energy_label = original_label
        return s

    def new_label(self, current_label):
        return min(current_label + 1, self.model.energy_label_max)

    def upgrade_cost(self, label_improvement):
        return label_improvement * self.model.cost_per_label_step

    # ---------------- Agent decision step ----------------

    def step(self):
        """
        Decide between:
        - stay
        - stay_upgrade
        - move
        - move_upgrade

        Only stores the decision; model applies moves and upgrades.
        """
        self.has_moved = 0
        self.has_upgraded = 0
        self.decision_type = None
        self._stay_upgrade_info = None
        self._move_info = None
        self._move_upgrade_info = None

        # ---- Option 1: stay ----
        current = self.current_house()
        stay_satisfaction = self.satisfaction(current)

        # ---- Option 2: stay & upgrade ----
        stay_upgrade_satisfaction = float("-inf")
        new_label_stay = self.new_label(current.energy_label)
        improvement_stay = new_label_stay - current.energy_label

        if improvement_stay > 0:
            cost_stay = self.upgrade_cost(improvement_stay)
            if cost_stay <= self.financial_capacity:
                stay_upgrade_satisfaction = self.satisfaction(
                    current, override_label=new_label_stay
                )
                self._stay_upgrade_info = (new_label_stay, cost_stay)

        # ---- Eligible houses in vacancy pool ----
        eligible = []
        for h in self.model.vacancy_pool():
            if h.house_id == self.current_house_id:
                continue
            if h.energy_label < self.min_acceptable_label:
                continue
            if h.market_value > self.financial_capacity:
                continue
            eligible.append(h)

        # ---- Option 3: move (no upgrade) ----
        move_satisfaction = float("-inf")
        best_house_id = None

        if eligible:
            best_score = float("-inf")
            best_house = None
            for h in eligible:
                s = self.satisfaction(h)
                if s > best_score:
                    best_score = s
                    best_house = h
            move_satisfaction = best_score
            best_house_id = best_house.house_id
            self._move_info = best_house_id

        # ---- Option 4: move & upgrade ----
        move_upgrade_satisfaction = float("-inf")

        if best_house_id is not None:
            target = self.model.houses[best_house_id]
            new_label_move = self.new_label(target.energy_label)
            improvement_move = new_label_move - target.energy_label
            cost_move = self.upgrade_cost(improvement_move)

            if improvement_move > 0 and cost_move <= self.financial_capacity \
               and target.market_value <= self.financial_capacity:
                move_upgrade_satisfaction = self.satisfaction(
                    target, override_label=new_label_move
                )
                self._move_upgrade_info = (best_house_id, new_label_move, cost_move)

        # ---- Choose best option ----
        scores = {
            "stay": stay_satisfaction,
            "stay_upgrade": stay_upgrade_satisfaction,
            "move": move_satisfaction,
            "move_upgrade": move_upgrade_satisfaction,
        }

        best_option = max(scores, key=scores.get)
        if scores[best_option] == float("-inf"):
            best_option = "stay"

        self.decision_type = best_option


# ============================================================
# Model
# ============================================================

class HousingUpgradeModel(mesa.Model):
    def __init__(
        self,
        n_homeowners,
        n_houses,
        energy_label_min=1,
        energy_label_max=7,
        market_value_min=150000,
        market_value_max=600000,
        cost_per_label_step=10000,
        seed=None,
    ):
        super().__init__()

        if seed is not None:
            np.random.seed(seed)
            import random as _r
            _r.seed(seed)

        self.energy_label_min = energy_label_min
        self.energy_label_max = energy_label_max
        self.market_value_min = market_value_min
        self.market_value_max = market_value_max
        self.cost_per_label_step = cost_per_label_step

        self.schedule = RandomActivation(self)

        # -------- Create house agents --------
        self.houses = {}       # house_id -> HouseAgent
        self.homeowners = []   # list of HomeownerAgent

        for h_id in range(n_houses):
            el = np.random.randint(self.energy_label_min, self.energy_label_max + 1)
            mv = np.random.uniform(self.market_value_min, self.market_value_max)
            house = HouseAgent(self, h_id, el, mv)
            self.houses[h_id] = house
            self.schedule.add(house)

        # -------- Create homeowners and assign houses --------
        house_ids = list(self.houses.keys())
        np.random.shuffle(house_ids)

        for i in range(n_homeowners):
            hid = house_ids[i]  # assume n_homeowners <= n_houses
            house = self.houses[hid]

            pref_energy = np.random.uniform(0.3, 0.7)
            pref_value = 1.0 - pref_energy
            financial_capacity = np.random.uniform(80000, 200000)
            min_label = np.random.randint(self.energy_label_min, self.energy_label_max + 1)

            homeowner = HomeownerAgent(
                self,
                current_house_id=hid,
                financial_capacity=financial_capacity,
                pref_energy=pref_energy,
                pref_value=pref_value,
                min_acceptable_label=min_label,
            )

            house.owner_id = homeowner.unique_id

            self.schedule.add(homeowner)
            self.homeowners.append(homeowner)

    # vacancy pool as a normal method (no decorator)
    def vacancy_pool(self):
        result = []
        for h in self.houses.values():
            if h.is_vacant():
                result.append(h)
        return result

    def step(self):
        """
        1) All agents do step():
           - HouseAgent: does nothing
           - HomeownerAgent: chooses decision_type
        2) Resolve competition for vacant houses
        3) Apply moves and upgrades
        """
        # Phase 1: agent decisions
        self.schedule.step()

        # Phase 2: competition for vacant houses
        competition = {}  # house_id -> list of HomeownerAgent

        for agent in self.schedule.agents:
            if not isinstance(agent, HomeownerAgent):
                continue

            if agent.decision_type == "move":
                target_id = agent._move_info
            elif agent.decision_type == "move_upgrade":
                if agent._move_upgrade_info is not None:
                    target_id = agent._move_upgrade_info[0]
                else:
                    target_id = None
            else:
                target_id = None

            if target_id is None:
                continue

            target_house = self.houses[target_id]
            if target_house.is_vacant():
                competition.setdefault(target_id, []).append(agent)

        # Determine winners (highest financial_capacity)
        winners = {}  # house_id -> winning agent
        for house_id, candidates in competition.items():
            best_agent = None
            best_cap = float("-inf")
            for a in candidates:
                if a.financial_capacity > best_cap:
                    best_cap = a.financial_capacity
                    best_agent = a
            winners[house_id] = best_agent

        # Phase 3: apply moves
        for house_id, winner in winners.items():
            old_house = self.houses[winner.current_house_id]
            new_house = self.houses[house_id]

            old_house.owner_id = None
            new_house.owner_id = winner.unique_id
            winner.current_house_id = house_id
            winner.has_moved = 1

        # Phase 4: apply upgrades and update budgets
        for agent in self.schedule.agents:
            if not isinstance(agent, HomeownerAgent):
                continue

            # stay + upgrade
            if agent.decision_type == "stay_upgrade" and agent._stay_upgrade_info is not None:
                new_label, cost = agent._stay_upgrade_info
                house = self.houses[agent.current_house_id]
                house.energy_label = new_label
                agent.financial_capacity -= cost
                agent.has_upgraded = 1

            # move + upgrade (only if move succeeded)
            if agent.decision_type == "move_upgrade" and agent.has_moved \
               and agent._move_upgrade_info is not None:
                target_id, new_label, cost = agent._move_upgrade_info
                house = self.houses[target_id]
                house.energy_label = new_label
                agent.financial_capacity -= cost
                agent.has_upgraded = 1


# ============================================================
# Quick smoke test
# ============================================================

model = HousingUpgradeModel(
    n_homeowners=50,
    n_houses=60,
    seed=42,
)

for t in range(5):
    model.step()
    moves = sum(isinstance(a, HomeownerAgent) and a.has_moved for a in model.schedule.agents)
    upgrades = sum(isinstance(a, HomeownerAgent) and a.has_upgraded for a in model.schedule.agents)
    print(
        "Step", t,
        "| moves:", moves,
        "| upgrades:", upgrades,
        "| vacant:", len(model.vacancy_pool())
    )


Mesa version: 3.0.3
Step 0 | moves: 0 | upgrades: 36 | vacant: 10
Step 1 | moves: 0 | upgrades: 31 | vacant: 10
Step 2 | moves: 0 | upgrades: 25 | vacant: 10
Step 3 | moves: 0 | upgrades: 18 | vacant: 10
Step 4 | moves: 0 | upgrades: 14 | vacant: 10


/var/folders/d5/blnkv71j1q3c6ggksh9jdk1h0000gn/T/ipykernel_76439/1016438042.py:248: DeprecationWarning: The time module and all its Schedulers are deprecated and will be removed in Mesa 3.1. They can be replaced with AgentSet functionality. See the migration guide for details. https://mesa.readthedocs.io/latest/migration_guide.html#time-and-schedulers
  self.schedule = RandomActivation(self)


In [3]:
print("Mesa version:", mesa.__version__)

model = HousingUpgradeModel(
    n_homeowners=50,
    n_houses=60,
    seed=42,
)

for t in range(5):
    model.step()
    print(
        f"Step {t}: moves={sum(a.has_moved for a in model.homeowners)}, "
        f"upgrades={sum(a.has_upgraded for a in model.homeowners)}, "
        f"vacant={len(model.vacant_houses)}"
    )



Mesa version: 3.0.3


AttributeError: module 'mesa' has no attribute 'AgentSet'

In [5]:
for t in range(5):
    model.step()
    moves = sum(a.has_moved for a in model.schedule.agents)
    upgrades = sum(a.has_upgraded for a in model.schedule.agents)
    avg_label = sum(h.energy_label for h in model.houses.values()) / len(model.houses)
    print(
        f"Step {t}: moves={moves}, upgrades={upgrades}, "
        f"vacant={len(model.vacancy_pool)}, avg_label={avg_label:.2f}"
    )



Step 0: moves=1, upgrades=8, vacant=10, avg_label=5.92
Step 1: moves=1, upgrades=6, vacant=10, avg_label=6.02
Step 2: moves=0, upgrades=5, vacant=10, avg_label=6.10
Step 3: moves=0, upgrades=2, vacant=10, avg_label=6.13
Step 4: moves=0, upgrades=1, vacant=10, avg_label=6.15
